# **(ADD THE NOTEBOOK NAME HERE)**

## Objectives

* Write your notebook objective here, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write down which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [54]:
import os
current_dir = os.getcwd()
current_dir

'c:\\DA Dev'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [55]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [56]:
current_dir = os.getcwd()
current_dir

'c:\\'

In [57]:
from pathlib import Path # load the pathlib library

import pandas as pd # load the pandas library
df = pd.read_csv('data/raw/insurance.csv') # read the csv file into a dataframe
df.head() # displays the first first few rows of dataframe
print(df.shape)  # look at shape of dataframe


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/insurance.csv'

# Section 1

Section 1 content

Import Libraries

In [ ]:
import numpy as np # load the numpy library
df.dtypes # check the data types of each column

---

Listing all the variable 

In [ ]:
df.describe(include='all') # get summary statistics of dataframe

Check for missing values

In [ ]:
#check for missing values
df.isnull().sum()


check for duplicates

In [ ]:
#check for duplicates
duplicate = df.duplicated().sum()
duplicate

Remove Duplicates and checking

In [ ]:
#remove duplicates
df = df.drop_duplicates().reset_index(drop=True)

duplicatech = df.duplicated().sum() #check that duplicates are gone
duplicatech

Check for NAN

In [ ]:
#check for nans
df.isna().sum() #



There are no NANs in the data set

Noramlise the data to make sure that all the categories will be consistent


Expect that there are Male and female are recorded corectly for sex and the spellings and spaces are correct
Expect that the regions are spelt consistently and spaces etc are consistent
Expect that the smoking is Yes or no and the spelling are consistent


In [ ]:
# check male and female is spelt correctly 
df["sex"] = df["sex"].str.strip().str.lower()


# check region is spelt correctly 
df["region"] = df["region"].str.strip().str.lower() #  can also do this  print(df['region'].unique()) and id all unique don't need to strip


# check smoking is set to yes or no
df["smoker"] = df["smoker"].str.strip().str.lower()






# Values count

In [ ]:
# value counts
df["sex"].value_counts()




In [ ]:
df["region"].value_counts()


In [ ]:
df["smoker"].value_counts()


In [ ]:
df["children"].value_counts()


In [ ]:
df["age"].value_counts()


In [ ]:
df["bmi"].value_counts()


In [ ]:
df["charges"].value_counts()

Histrogram of all variables to show shapes

In [ ]:
df.hist()

#Summary 

Age is an uniform distribution there is a single peak to the left

BMI has a normal distribution according with some high values greater than 45.

Charges are strongly skewed to the right - most charges appear to be under 15K according to the histrogram

For childrem most people have three or under

In [ ]:
## Since 'bmi' is a continuous variable, we can categorize it into groups for better analysis.

bmi_bins = [0, 18.5, 24.9, 29.9, 34.9, 39.9, 100]
bmi_labels = ['Underweight', 'Normal weight', 'Overweight', 'Obesity I', 'Obesity II', 'Obesity III']
df['bmi_category'] = pd.cut(df['bmi'], bins=bmi_bins, labels=bmi_labels)
df['bmi_category'].value_counts()


In [ ]:
df['bmi_category'].value_counts().sort_index()

In [ ]:
grouped = df.groupby(['bmi_category', 'smoker'], observed=True)['charges'].mean()
print(grouped)

In [ ]:
# Group and calculate mean charges by bmi_category and smoker
pivot = df.groupby(['bmi_category', 'smoker'], observed=True)['charges'].mean().unstack()
	
# 2. Calculate difference and percent difference (numeric)
pivot['difference'] = pivot['yes'] - pivot['no']
pivot['percent_diff'] = (pivot['difference'] / pivot['no']) * 100
	
# 3. Format the currency columns
pivot['no'] = pivot['no'].map(lambda x: f"${x:,.0f}")
pivot['yes'] = pivot['yes'].map(lambda x: f"${x:,.0f}")
pivot['difference'] = pivot['difference'].map(lambda x: f"£{x:,.0f}")

# 4. Format the percent difference column
pivot['percent_diff'] = pivot['percent_diff'].map(lambda x: f"{x:.1f}%")
print(pivot)


Focus on 'BMI' and 'Age' as a factor in Healthcare cost

In [ ]:
grouped = df.groupby(['bmi_category', 'age'], observed=True)['charges'].mean().reset_index()

grouped['charges'] = grouped['charges'].map(lambda x: f"£{x:,.0f}")

print(grouped)


Convert from Long to Wide format for better reading and understanding of Dataset

In [ ]:
pivot_table = pd.pivot_table(
	    df,
	    values='charges',               # Column to aggregate
	    index='bmi_category',          # Rows
	    columns='age',                 # Columns
	    aggfunc='mean',                # Aggregation function
	    fill_value=0                   # Replace missing values with 0
).reset_index().rename_axis(mapper=None, axis=1)
age_cols = pivot_table.columns.drop('bmi_category')

pivot_table[age_cols] = pivot_table[age_cols].map(lambda x: f"£{x:,.0f}")
print(pivot_table)

In [ ]:
# Create age bands
def age_band(x):
    if x < 18: return "<18"
    elif x <= 25: return "18-25"
    elif x <= 34: return "26-34"
    elif x <= 49: return "35-49"
    elif x <= 50: return "50-69"
    elif x <= 70: return "70-79"
    return "80+"
df["age_band"] = df["age"].apply(age_band)
df.head()

# Statistical testing and creation of outlier group
**ANOVA**
Tests was run to see if there is a statistically signficant differrence in mean charges between at least one of the child counts. The result tells us there is a difference but not where. Within our dataset most people have 0, 1, or 2 children. Very few have 4-5 so small sample outliers in this group may be driving this. You can see there are only 25 people with 4 children or more. ANOVA indicates differences, but the largest effects are in very small groups (4–5 children). Small sample sizes may distort averages what if the people in this smaller group are mostly non smokers and/or are younger than the average? The ANOVA only checked: Do charges differ by number of children? It didn’t control for other factors like:

Smoker status (huge driver of charges!)
Age (older = higher charges)
BMI
So, if groups with 4–5 children just happen to have fewer smokers and younger people, their lower charges might have nothing to do with family size at all!

To control this need to us multiple regression (preferred). Model charges as a function of children, smoker, age, BMI, region. The coefficients for children in this test now show the effect of family size after controlling for age, smoker, BMI, and region.

In this regression shows: once you control for smoker and age, the effect of children is weak and inconsistent. Children coefficients (adjusted for age, smoker, region)

These numbers show the extra cost compared to the reference group (0 children):

1 child: +$387, not significant (p = 0.359)
2 children: +$1633, significant (p < 0.001)
3 children: +$961, not significant (p = 0.080, borderline)
4 children: +$2936, significant (p = 0.018)
5 children: +$1105, not significant (p = 0.448)

So after adjusting for smoking, age, and region, most child groups are not consistently different. Only 2 children and 4 children stand out as significantly higher charges. No children is not clearly more expensive the only clear differences are small bumps at 2 and 4 kids. The real heavy hitters are smoker status and age, not children.


In [ ]:
# ANOVA test to see if there are significant differences in charges based on number of children
import scipy.stats as stats

# Run ANOVA
groups = [df.loc[df["children"]==n, "charges"] for n in sorted(df["children"].unique())]
f_stat, p_val = stats.f_oneway(*groups)

print("F-statistic:", f_stat)
print("p-value:", p_val)


In [ ]:
import statsmodels.formula.api as smf
model = smf.ols('charges ~ C(children) + C(smoker) + age + bmi + C(region)', data=df).fit() # multiple regression model
model.summary() # display the summary statistics of the regression model


In [ ]:
import statsmodels.formula.api as smf

# Model 1: Smoking + Age
model1 = smf.ols("charges ~ C(smoker) + age", data=df).fit()

# Model 2: Smoking + BMI
model2 = smf.ols("charges ~ C(smoker) + bmi", data=df).fit()

print("Smoking + Age R²:", model1.rsquared)
print("Smoking + BMI R²:", model2.rsquared)

Section 2 content

BMI and smoking compared to Age and smoking are what I look at below; Both models are strong, but age adds more predictive power than BMI when combined with smoking. In practical terms: Being older consistently increases charges. BMI matters, but its effect is weaker and less consistent across the dataset compared to age. However in the below raw BMI assumes the effect of BMI is linear (i.e.) each unit increase adds the same change in charges. In reality, insurance risk often jumps when you cross thresholds (25, 30, 35…). Bands let you capture those non-linear effects:

Underweight
Healthy
Overweight
Obese Class 1
Obese Class 2
Obese Class 3

In [ ]:
import statsmodels.formula.api as smf

# Model 1: Smoking + Age
model1 = smf.ols("charges ~ C(smoker) + age", data=df).fit()

# Model 2: Smoking + BMI
model2 = smf.ols("charges ~ C(smoker) + bmi", data=df).fit()

print("Smoking + Age R²:", model1.rsquared)
print("Smoking + BMI R²:", model2.rsquared)

# Results of the statistical tests conducted below show:

**smoking:**

- Coefficient ~ +$23,900
- p < 0.001 Smoking is the single largest driver of charges.

**Age bands (baseline = 18–25):**

- 26–34: +$1,415 (significant)
- 35–49: +$4,314 (significant)
- 50–69: +$8,143 (significant)
- 70–79: +$9,184 (significant) Clear stepwise increase across age bands.

**BMI categories (baseline = Class 1 Obesity):**

- Class 2 Obesity: +$1,291 (significant)
- Class 3 Obesity: +$1,095 (not significant)
- Healthy Weight: –$4,085 (significant)

**Overweight: –$3,256 (significant)**

- Underweight: –$5,796 (significant) Healthy/Overweight/Underweight all cost less than obese.

**Conclusion**

- Smoking dominates (adds ~$24k regardless of age or BMI).
- Age adds a strong, stepwise effect (costs climb steadily after 35).
- BMI bands show clear differences: being obese adds thousands compared to healthy/overweight.

Note: Using bands makes the effects easier to present:
- Non-smokers in their 20s with healthy BMI average ~$5–6k.
- Smokers in their 50s with obesity average ~$30k+.

In [ ]:
# Age Band smoking & BMI Test
# I have already created 'age_band' and 'bmi_category' columns
model_bands = smf.ols("charges ~ C(smoker) + C(age_band) + C(bmi_category)", data=df).fit()

print(model_bands.summary())
print("R² with Smoking + Age bands + BMI bands:", model_bands.rsquared)

# Smoking differences between regions do we have a region with more smokers?

I look at that below and found there isn’t strong statistical evidence that some regions “have more smokers.” Which means regional charge differences are unlikely to be explained by smoking distribution alone.

In [ ]:
from scipy.stats import chi2_contingency

# Contingency table
table = pd.crosstab(df['region'], df['smoker'])
chi2, p, dof, expected = chi2_contingency(table)

print(table)
print("Chi-square:", chi2, "p-value:", p)

In the below we identify outlier (highest charge cases) in top Q1 quantile and create a tag so they can be filtered in Power BI.

In [ ]:
# Example: charges column in insurance dataset
Q1 = df['charges'].quantile(0.25)
Q3 = df['charges'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# filter outliers (High charges only)
outliers = df[(df['charges'] > upper_bound)]

# I then created a new column to tag the records so they can be filtered out in Power BI a g later

def high_cost_case(x):
    if x >= 40000:   # threshold you chose
        return "High Cost"
    else:
        return "Normal"

df['cost_group'] = df['charges'].apply(high_cost_case)

We look at BMI and region below to see if there are any regional differences that are significant. Our hypotheses was that therew woule be no regional differences.

In [ ]:
pd.crosstab(df['region'], df['bmi_category']) # check distribution of bmi categories across regions

In the above we could see there was a regional difference in the Southeast in category Obesity III. We did further tests to see if this was a significant difference below. The results below show a tthe difference is significant. Results came back significant. as the the distribution of BMI categories differs significantly across regions. Some regions  do have a different BMI risk mix. Looking at our crosstab, the Southeast had a much higher Class 3 Obesity count (55 people vs 8–15 elsewhere). That is probably what drove this big Chi-square.

In [ ]:
chi2, p, dof, expected = chi2_contingency(pd.crosstab(df['region'], df['bmi_category'])) # Chi-square test for BMI category across regions
print("Chi-square:", chi2, "p-value:", p) # check p-value

In [ ]:
df.to_csv('data\processed\insurance_cleaned.csv', index=False) # export cleaned dataframe to csv

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

# Push files to Repo

---